# Green Manufacturing

In [1]:
import sys
import os

sys.path.append('/Users/janinedevera/Documents/ADB/Climate/eemrio/src')
os.chdir('/Users/janinedevera/Documents/ADB/Climate/eemrio/')

In [2]:
import pandas as pd
import numpy as np
import duckdb
from mriomain import mrio

## I. Load Data

In [182]:
input_mrio = './data/mrio.parquet'
input_ee = './data/ee.parquet'
countries_mapping = pd.read_excel('./data/raw/countries.xlsx').drop_duplicates(subset=['iso_a3'], keep='last')
sectors_mapping = pd.read_csv('./data/raw/sectors.csv')

years = [2017, 2018, 2019, 2020, 2021, 2022]

In [4]:
# column labels
df = duckdb.sql(f"SELECT * FROM '{input_ee}'").df()
labels = df.columns[4:2559].to_frame(index=False)[0].str.split("_", expand=True)
labels = labels.set_axis(['country', 'mrio_sector'], axis=1)

# mapping: ghg sectors
ghg_sectors = sorted(df['sector'].unique().tolist())

# mapping: mrio sectors
sectors_map = sectors_mapping.iloc[:, 0:3]
sectors_map['mrio_sector'] = 'c' + sectors_map['c_ind'].astype(str)
sectors_map = sectors_map.rename(columns={'c_ind': 'sector_no', 'c_name': 'sector_name', 'c_name_short': 'sector_name_short'})
agg_map = sectors_mapping.iloc[:, [0, 5]]

# mapping: regions
region_map = countries_mapping.iloc[:, [8,12]]

# industrial sectors 
ind_sectors = [f"c{i}" for i in range(3, 16+1)]

In [185]:
# baci data
baci_years = [filename for filename in os.listdir('./data/raw/baci/') if filename.startswith("BACI_")]

country_codes = pd.read_csv('./data/raw/baci/country_codes_V202401b.csv')
product_codes = pd.read_csv('./data/raw/baci/product_codes_HS17_V202401b.csv')

# adb mapping
mrio_mapping = pd.read_excel('./data/raw/product-correspondence.xlsx').drop_duplicates(subset=['HS17'], keep='first')
countries_mapping = pd.read_excel('./data/raw/countries.xlsx')

# industrial sectors 
ind_sectors = [f"c{i}" for i in range(3, 16+1)]

In [ ]:
# combine data for all years
# temp_list = []

# for filename in baci_years:
#     filepath = os.path.join('./data/raw/baci/', filename)
#     df = pd.read_csv(filepath)  
#     temp_list.append(df) 

# baci_raw = pd.concat(temp_list, ignore_index=True).drop(columns=['q'])
# baci_raw.columns = ['year', 'exporter', 'importer', 'product', 'value']
# baci_raw['value'] = pd.to_numeric(baci_raw['value'])

In [54]:
# mrio value added data
gva = pd.DataFrame()
for year in years: 
    temp = mrio.MRIO(f'{input_mrio}', year)
    gva[year] = temp.va.data

gva = pd.concat([labels, gva], axis=1)
gva = pd.melt(gva, id_vars=['country', 'mrio_sector'], value_vars=[2017, 2018, 2019, 2020, 2021, 2022]).rename(columns={"value": "total_gva", "variable": "year"})

In [ ]:
gva_2022 = gva[gva['year'] == 2022]

## II. Process 

### A. MRIO

In [200]:
# get unique country-sector combinations 
mrio_list = gva[['country', 'mrio_sector']].drop_duplicates()

# map to metadata
mrio_list = pd.merge(mrio_list, sectors_map, on='mrio_sector')
mrio_list = pd.merge(mrio_list, agg_map, left_on='sector_no', right_on='c_ind').drop(columns=['c_ind'])
mrio_list = pd.merge(mrio_list, region_map, left_on='country', right_on='code', how='left').drop(columns=['code'])

### B. BACI

In [6]:
filepath = os.path.join('./data/raw/baci/', 'BACI_HS17_Y2022_V202401b.csv')
df = pd.read_csv(filepath).drop(columns=['q'])

df.columns = ['year', 'exporter', 'importer', 'product', 'value']
df['value'] = pd.to_numeric(df['value'])

In [8]:
# aggregate importers 
baci_agg = (
    df
    .groupby(['year', 'exporter', 'product'])
    .agg('sum').reset_index()
    .drop(columns=['importer'])
    .rename(columns={"value": "export_val"})
)

# map to country and product names
baci = pd.merge(baci_agg, country_codes, left_on='exporter', right_on='country_code').drop(columns=['country_code'])
baci = pd.merge(baci, product_codes, left_on='product', right_on='code').drop(columns=['code'])

# extract mrio countries
countries_mapping = countries_mapping[['iso_a3', 'code', 'name_adb', 'region_wb', 'mrio', 'region']]

baci_mrio = (pd.merge(baci, countries_mapping, left_on='country_iso3', right_on='iso_a3'))
baci_mrio = baci_mrio[baci_mrio['mrio'].notna()].reset_index()

In [16]:
# load UNIDO green products list 
green_list = pd.read_excel('./data/raw/UNIDO-green-product-list-2022.xlsx').iloc[: , 1:]
#green_list = green_list['HS CODE'].to_list()

In [29]:
# identify green products
baci_merge = (pd.merge(baci_mrio, green_list, left_on='product', right_on='HS CODE', how='left'))
baci_merge = baci_merge.drop(['index', 'HS CODE', 'Product', 'Source', 'ISIC3', 'ISIC4'], axis=1)
baci_merge['EA'] = baci_merge['EA'].fillna('NG')


In [32]:
baci_merge['EA'].value_counts()

EA
NG    295861
PC      6970
EC      3732
RC      3017
RE      2248
Name: count, dtype: int64

In [41]:
baci_full = (pd.merge(baci_merge, mrio_mapping, left_on='product', right_on='HS17')).drop(columns=['HS2002'])
#baci_full['MRIOsec'].value_counts()

MRIOsec
c4     44037
c9     43673
c12    35062
c13    30137
c3     26604
c14    24033
c1     14706
c16    10638
c11     9132
c7      8462
c15     7755
c10     7260
c2      5195
c5      4254
c6      3278
c8      1050
c34      624
c17      187
c30      173
Name: count, dtype: int64

### C. Calculate green exports share

In [100]:
# exports: green shares per sector
sector_df = (
    baci_full[['year', 'country_iso3', 'code', 'name_adb', 'region', 'MRIOsec', 'MRIOdesc', 'EA', 'export_val']]
    .groupby(['year', 'country_iso3', 'code', 'name_adb', 'region', 'MRIOsec', 'MRIOdesc', 'EA'])
    .agg('sum')
    .reset_index()
)

# get unique observations (country+sector) and green types
sector_df_unique = sector_df[['year', 'country_iso3', 'code', 'name_adb', 'region', 'MRIOsec', 'MRIOdesc']].drop_duplicates()
green_types = sector_df['EA'].unique().tolist()
green_industries = sector_df['MRIOsec'].unique().tolist()

# merge green types to sector df 
# sector_df_full = sector_df_unique.assign(key=1).merge(pd.DataFrame({'EA': green_types, 'key': 1}), on='key').drop('key', axis=1)
# sector_df_full = sector_df_full.merge(sector_df, on=['year', 'country_iso3', 'code', 'name_adb', 'region', 'MRIOsec', 'MRIOdesc', 'EA'], how='left')
# sector_df_full['export_val'] = sector_df_full['export_val'].fillna(0)

In [260]:
sector_df_full = mrio_list.assign(key=1).merge(pd.DataFrame({'EA': green_types, 'key': 1}), on='key').drop('key', axis=1)

In [261]:
sector_df_full = (
    sector_df_full.merge(
    sector_df, 
    left_on=['country', 'region',  'mrio_sector', 'EA'],
    right_on=['code', 'region', 'MRIOsec', 'EA'], 
    how='left'
    )
    .drop(columns=['year', 'country_iso3', 'code', 'name_adb', 'MRIOsec', 'MRIOdesc'])
    )

sector_df_full['export_val'] = sector_df_full['export_val'].fillna(0)

In [263]:
# calculate shares
sector_df_full['total_exports'] = sector_df_full.groupby(['country', 'mrio_sector', 'sector_no', 'sector_name', 'sector_name_short', 'c15_name', 'region'])['export_val'].transform('sum')
sector_df_full['export_share'] = sector_df_full['export_val'] / sector_df_full['total_exports']
sector_df_full['export_share'] = sector_df_full['export_share'].fillna(0)

In [285]:
sector_df_full['original_index'] = sector_df_full.index
grouped_df = sector_df_full.groupby(['country', 'mrio_sector', 'sector_no', 'sector_name', 'sector_name_short', 'c15_name'])

def adjust_export_share(group):
    total_export_share = group['export_share'].sum()

    if total_export_share == 1:
        return group

    if total_export_share == 0:
        group.loc[group['EA'] == 'NG', 'export_share'] = 1

    return group

sector_df_full_adjusted = grouped_df.apply(adjust_export_share).reset_index(drop=True)
sector_df_full_adjusted = sector_df_full_adjusted.sort_values(by='original_index').drop(columns=['original_index']).reset_index(drop=True)


/var/folders/rd/n9w0gpv53y72x5k9wk3hp63w0000gn/T/ipykernel_1652/2665987089.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sector_df_full_adjusted = grouped_df.apply(adjust_export_share).reset_index(drop=True)


In [290]:
# domestic: green shares per sector
sector_dom = pd.merge(sector_df_full_adjusted, gva_2022, on=['country', 'mrio_sector'], how='right')
sector_dom['va_val'] = sector_dom['export_share'] * sector_dom['total_gva']

In [316]:
sector_dom_sorted = sector_dom.sort_values(by=['country', 'EA', 'mrio_sector']).reset_index(drop=True)

In [297]:
sector_dom.to_excel('./data/raw/output_shares.xlsx')

# MRIO Disaggregation

## I. 

In [355]:
# sort values by EA type
country_order = sector_dom['country'].unique()
ea_order = sector_dom['EA'].unique()

sector_dom['country'] = pd.Categorical(sector_dom['country'], categories=country_order, ordered=True)
sector_dom['EA'] = pd.Categorical(sector_dom['EA'], categories=ea_order, ordered=True)
sector_dom_sorted = sector_dom.sort_values(by=['country', 'EA', 'sector_no']).reset_index(drop=True)

# get export share row vector
export_shares = sector_dom_sorted['export_share'].to_numpy()

In [423]:
export_shares_reshape = export_shares.reshape(-1, 35)
diagonal_matrices = np.array([np.diag(export_shares_reshape[i]) for i in range(export_shares_reshape.shape[0])])

In [432]:
diagonal_matrices = diagonal_matrices.reshape(73, 5, 35, 35)